# Setup

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier
import optuna
from optuna.integration import CatBoostPruningCallback

In [7]:
import matplotlib
%matplotlib inline

# Optuna

In [2]:
def load_data():
    data = pd.read_csv(
        'https://raw.githubusercontent.com/antbartash/australian_rain/main/data/data_transformed.csv',
        index_col=0
    )
    X, y = data.drop(columns=['RainTomorrow', 'RainToday']), data['RainTomorrow']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    for column in ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']:
        X_train[column] = X_train[column].astype(np.float32).fillna(-1).apply(lambda x: str(x))
        X_test[column] = X_test[column].astype(np.float32).fillna(-1).apply(lambda x: str(x))
    return X_train, X_test, y_train, y_test

In [3]:
def objective(trial):
    PARAMS = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 0.5),
        'depth': trial.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0, 100.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 100.0), # CPU only
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 100.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 4)
    }
    X_train, X_valid, y_train, y_valid = load_data()
    pruning_callback = CatBoostPruningCallback(trial, 'AUC')
    model = CatBoostClassifier(
        cat_features=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'],
        custom_metric='AUC',
        random_state=42, verbose=False, task_type='CPU'
    )
    model.set_params(**PARAMS)
    model.fit(X_train, y_train, 
              eval_set=(X_valid, y_valid),
              callbacks=[pruning_callback]
    )
    score = np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='roc_auc'))
    return score

In [4]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials = 5, # the random sampling is used instead of the TPE algorithm until the given number of trials finish in the same study
    n_ei_candidates = 24, # number of candidate samples used to calculate the expected improvement.
    multivariate = True, # multivariate TPE when suggesting candidates; default: False
    seed = 42
)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=100, interval_steps=20),
    direction='maximize',
    sampler=sampler,
    storage='sqlite:///db.sqlite3'
)
study.optimize(
    objective, n_trials = 100,
    timeout = 3600, # in seconds
    n_jobs = 1,
    show_progress_bar = True
)

/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2023-12-29 09:53:04,726] A new study created in RDB with name: no-name-568d9569-4f33-4c33-a3b3-53cde451dcff


  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_42/4059298355.py:13: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, 'AUC')


[I 2023-12-29 09:55:08,071] Trial 0 finished with value: 0.8797334698575231 and parameters: {'n_estimators': 406, 'learning_rate': 0.47535720249065166, 'depth': 8, 'l2_leaf_reg': 59.86584841970366, 'random_strength': 15.601864044243651, 'bagging_temperature': 15.599452033620265, 'grow_policy': 'Depthwise', 'scale_pos_weight': 2.8033450352296265}. Best is trial 0 with value: 0.8797334698575231.


/tmp/ipykernel_42/4059298355.py:13: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, 'AUC')


...

[I 2023-12-29 10:50:56,221] Trial 23 finished with value: 0.90060082050988 and parameters: {'n_estimators': 570, 'learning_rate': 0.16965462869310532, 'depth': 5, 'l2_leaf_reg': 24.93684743624295, 'random_strength': 94.65568845231142, 'bagging_temperature': 1.2598103281196344, 'grow_policy': 'SymmetricTree', 'scale_pos_weight': 2.737266650319726}. Best is trial 17 with value: 0.9010773253239965.


/tmp/ipykernel_42/4059298355.py:13: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, 'AUC')


[I 2023-12-29 10:51:59,078] Trial 24 finished with value: 0.8956000362364884 and parameters: {'n_estimators': 541, 'learning_rate': 0.15019789086678176, 'depth': 3, 'l2_leaf_reg': 27.520778004197876, 'random_strength': 93.9588142253623, 'bagging_temperature': 19.271766709381996, 'grow_policy': 'SymmetricTree', 'scale_pos_weight': 3.3852177536046892}. Best is trial 17 with value: 0.9010773253239965.


/tmp/ipykernel_42/4059298355.py:13: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, 'AUC')


[I 2023-12-29 10:55:02,222] Trial 25 finished with value: 0.9018302574773327 and parameters: {'n_estimators': 722, 'learning_rate': 0.07139217176610062, 'depth': 8, 'l2_leaf_reg': 28.24988726549349, 'random_strength': 87.72871371560576, 'bagging_temperature': 54.029297770604856, 'grow_policy': 'SymmetricTree', 'scale_pos_weight': 1.1872960291692018}. Best is trial 25 with value: 0.9018302574773327.


In [5]:
print(f"Best Gini: {study.best_trial.value * 2 - 1}")
print(f"Best params: {study.best_trial.params}")

Best Gini: 0.8036605149546654
Best params: {'n_estimators': 722, 'learning_rate': 0.07139217176610062, 'depth': 8, 'l2_leaf_reg': 28.24988726549349, 'random_strength': 87.72871371560576, 'bagging_temperature': 54.029297770604856, 'grow_policy': 'SymmetricTree', 'scale_pos_weight': 1.1872960291692018}


# Plots

In [8]:
optuna.visualization.plot_edf(study)

In [9]:
optuna.visualization.plot_optimization_history(study)

In [10]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'learning_rate'])

In [12]:
optuna.visualization.plot_parallel_coordinate(study)

In [13]:
optuna.visualization.plot_param_importances(study)

In [14]:
optuna.visualization.plot_slice(study)

In [15]:
optuna.visualization.plot_timeline(study)

/tmp/ipykernel_42/2497081126.py:1: ExperimentalWarning:

plot_timeline is experimental (supported from v3.2.0). The interface can change in the future.

